Name: Patrick Ng  
Email: patng@ischool.berkeley.edu  
Class: W261-2  
Week: 03  
Date of submission: Feb 01, 2016

## HW3.0.

What is a merge sort? Where is it used in Hadoop?  
How is  a combiner function in the context of Hadoop?   
Give an example where it can be used and justify why it should be used in the context of this problem.  
What is the Hadoop shuffle?

TBD

## HW3.1 
Use Counters to do EDA (exploratory data analysis and to monitor progress)
Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?


User-defined Counters

Now, let’s use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 

### Mapper

In [15]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

# Skip the header line
sys.stdin.readline()

# input comes from STDIN (standard input)
for line in sys.stdin:
    fields = line.split(",")
    reason = fields[1].lower()
    
    if reason == "debt collection":
        counter = "debt"
    elif reason == "mortgage":
        counter = "mortgage"
    else:
        counter = "others"
        
    sys.stderr.write('reporter:counter:custom,' + counter + ',1\n')


Overwriting mapper.py


In [14]:
# Quick test
!head -n 20 Consumer_Complaints.csv  | python mapper.py

reporter:counter:custom,debt,1
reporter:counter:custom,debt,1
reporter:counter:custom,others,1
reporter:counter:custom,debt,1
reporter:counter:custom,others,1
reporter:counter:custom,debt,1
reporter:counter:custom,others,1
reporter:counter:custom,debt,1
reporter:counter:custom,others,1
reporter:counter:custom,debt,1
reporter:counter:custom,others,1
reporter:counter:custom,others,1
reporter:counter:custom,debt,1
reporter:counter:custom,mortgage,1
reporter:counter:custom,others,1
reporter:counter:custom,others,1
reporter:counter:custom,debt,1
reporter:counter:custom,debt,1
reporter:counter:custom,debt,1
NA


In [2]:
# upload input file to hdfs
!hdfs dfs -rm -f Consumer_Complaints.csv
!hdfs dfs -put Consumer_Complaints.csv

16/01/30 17:26:52 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 17:26:54 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
# Hadoop streaming command
!hdfs dfs -rm -r hw3.1
!hadoop jar $HADOOP_INSTALL/share/hadoop/tools/lib/hadoop-*streaming*.jar -D mapred.reduce.tasks=0 -file mapper.py -mapper mapper.py -input Consumer_Complaints.csv -output hw3.1

16/01/30 17:40:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3.1
16/01/30 17:40:12 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/30 17:40:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/hadoop-unjar6298878087895734314/] [] /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/streamjob2623231940027838210.jar tmpDir=null


### Result

![result](https://photos-3.dropbox.com/t/2/AAD96apEVb1NEyOodPWCtkdcs8K_w-nW4PJtqZe6LTUSCQ/12/15674996/png/32x32/1/_/1/2/Screenshot%202016-01-30%2017.42.35.png/EKi01gsYnS8gBygH/Hs_dhV-YD5vL1Ja5tyxj1tFInEhRb2H_uNNtVF96zBs?size=1024x768&size_mode=3)

## HW 3.2 

Analyze the performance of your Mappers, Combiners and Reducers using Counters

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux


Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 
Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 